In [1]:
1 + 2

3

In [ ]:
import ollama
import numpy as np
from numpy.linalg import norm


def get_embedding(text):
    response = ollama.embeddings(model="embeddinggemma:300m", prompt=text)
    return np.array(response["embedding"])


def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))


# 1. Generate embeddings
king = get_embedding("king")
man = get_embedding("man")
woman = get_embedding("woman")
queen = get_embedding("queen")

# 2. Perform the arithmetic: King - Man + Woman
# We are essentially "subtracting" the concept of masculinity
# and "adding" the concept of femininity to the concept of royalty.
masculinity_vector = man - woman
result_vector = king - masculinity_vector

# 3. Verify the result
similarity = cosine_similarity(king, queen)

print(f"Similarity between (King - Man + Woman) and Queen: {similarity:.4f}")

Similarity between (King - Man + Woman) and Queen: 0.7330


In [18]:
import json
import heapq
import os
import subprocess
from pathlib import Path
from typing import Iterable, List, Optional, Tuple, Union, cast

import numpy as np
import gguf

# Use an Ollama model reference, e.g. "qwen3:0.6b" or "gemma3:27B".
# This notebook reads the local Ollama GGUF blob directly (no HuggingFace downloads).

OLLAMA_MODEL = "translategemma:4b"

MANIFESTS_ROOT = Path.home() / ".ollama/models/manifests"
BLOBS_ROOT = Path.home() / ".ollama/models/blobs"


def _parse_ollama_model_ref(model: str) -> Tuple[str, str, str, str]:
    """Parse Ollama model reference into (host, namespace, repo_path, tag)."""
    model = model.strip()
    if not model:
        raise ValueError("Empty OLLAMA_MODEL")

    if ":" in model:
        repo_part, tag = model.rsplit(":", 1)
        tag = tag or "latest"
    else:
        repo_part, tag = model, "latest"

    parts = [p for p in repo_part.split("/") if p]
    if len(parts) >= 3:
        host = parts[0]
        namespace = parts[1]
        repo_path = "/".join(parts[2:])
    elif len(parts) == 2:
        host = "registry.ollama.ai"
        namespace = parts[0]
        repo_path = parts[1]
    else:
        host = "registry.ollama.ai"
        namespace = "library"
        repo_path = parts[0]

    return host, namespace, repo_path, tag


def _candidate_manifest_paths(model: str) -> List[Path]:
    host, namespace, repo_path, tag = _parse_ollama_model_ref(model)
    repo_parts = repo_path.split("/")

    candidates: List[Path] = []
    direct = MANIFESTS_ROOT / host / namespace
    for rp in repo_parts:
        direct = direct / rp
    direct = direct / tag
    candidates.append(direct)

    # Fallback: match any manifest that ends with /<repo_last>/<tag>.
    repo_last = repo_parts[-1]
    candidates.extend(MANIFESTS_ROOT.glob(f"**/{repo_last}/{tag}"))

    # De-duplicate preserving order
    seen: set[Path] = set()
    out: List[Path] = []
    for p in candidates:
        if p not in seen:
            seen.add(p)
            out.append(p)
    return out


def _resolve_ollama_manifest(model: str) -> Path:
    candidates = _candidate_manifest_paths(model)
    existing = [p for p in candidates if p.is_file()]
    if not existing:
        raise FileNotFoundError(
            "Could not find an Ollama manifest for model "
            f"{model!r} under {MANIFESTS_ROOT}. Tried:\n"
            + "\n".join(str(p) for p in candidates[:10])
        )
    if len(existing) == 1:
        return existing[0]
    existing.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    return existing[0]


def _resolve_ollama_gguf_blob(manifest_path: Path) -> Path:
    obj = json.loads(manifest_path.read_text(encoding="utf-8"))
    for layer in obj.get("layers", []):
        if layer.get("mediaType") == "application/vnd.ollama.image.model":
            digest = layer["digest"]  # e.g. sha256:...
            if not digest.startswith("sha256:"):
                raise ValueError(f"Unexpected digest format: {digest}")
            blob = BLOBS_ROOT / ("sha256-" + digest.split(":", 1)[1])
            if not blob.exists():
                raise FileNotFoundError(f"Resolved blob does not exist: {blob}")
            return blob
    raise ValueError(f"No model layer found in manifest: {manifest_path}")


def _assert_gguf_file(path: Path) -> None:
    with path.open("rb") as f:
        magic = f.read(4)
    assert magic == b"GGUF", f"Expected GGUF file, got magic={magic!r} at {path}"


def _tok_to_str(x: Union[str, bytes]) -> str:
    if isinstance(x, bytes):
        return x.decode("utf-8", errors="replace")
    return str(x)


def _field_to_str(field) -> Optional[str]:
    if not field:
        return None
    data = getattr(field, "data", None)
    if isinstance(data, list) and data:
        value = data[0]
        if isinstance(value, bytes):
            return value.decode("utf-8", errors="replace")
        return str(value)
    if isinstance(data, bytes):
        return data.decode("utf-8", errors="replace")
    if isinstance(data, (str, int, float)):
        return str(data)
    try:
        contents = field.contents()
    except Exception:
        return None
    if isinstance(contents, list) and contents:
        value = contents[0]
        if isinstance(value, bytes):
            return value.decode("utf-8", errors="replace")
        return str(value)
    if isinstance(contents, bytes):
        return contents.decode("utf-8", errors="replace")
    if isinstance(contents, (str, int, float)):
        return str(contents)
    return None


def _parse_chat_template_value(value: str) -> Optional[str]:
    s = value.strip()
    if not s or s.isdigit():
        return None
    if s[0] in "[{":
        try:
            obj = json.loads(s)
        except Exception:
            obj = None
        if isinstance(obj, dict):
            tmpl = obj.get("template")
            if tmpl is not None:
                return str(tmpl)
            return None
        if isinstance(obj, list):
            for item in obj:
                if (
                    isinstance(item, dict)
                    and item.get("name") == "default"
                    and "template" in item
                ):
                    return str(item["template"])
            for item in obj:
                if isinstance(item, dict) and "template" in item:
                    return str(item["template"])
            return None
    return s


def _looks_like_template(text: str) -> bool:
    t = text.strip()
    if not t:
        return False
    if t.isdigit():
        return False
    if "{{" in t and "Messages" in t:
        return True
    if "<start_of_turn>" in t or "<|im_start|>" in t:
        return True
    if "add_generation_prompt" in t:
        return True
    return False


def _get_chat_template_from_gguf(reader: gguf.GGUFReader) -> Optional[str]:
    field = reader.fields.get("tokenizer.chat_template")
    raw = _field_to_str(field)
    if raw:
        template = _parse_chat_template_value(raw)
        if template and _looks_like_template(template):
            return template

    for key, field in reader.fields.items():
        if "chat_template" not in key:
            continue
        raw = _field_to_str(field)
        if not raw:
            continue
        template = _parse_chat_template_value(raw)
        if template and _looks_like_template(template):
            return template

    for field in reader.fields.values():
        raw = _field_to_str(field)
        if not raw:
            continue
        template = _parse_chat_template_value(raw)
        if template and _looks_like_template(template):
            return template
    return None


def _get_chat_template_name_from_gguf(reader: gguf.GGUFReader) -> Optional[str]:
    field = reader.fields.get("tokenizer.chat_template")
    raw = _field_to_str(field)
    if not raw:
        return None
    name = raw.strip()
    if not name or name.isdigit():
        return None
    if name[0] in "[{":
        return None
    if _looks_like_template(name):
        return None
    return name


def _get_ollama_repo_template(
    template_name: str, repo_root: Optional[Path] = None
) -> Optional[str]:
    if not template_name:
        return None
    repo_root = repo_root or (Path.home() / "workspace/github.com/ollama/ollama")
    template_dir = repo_root / "template"
    if not template_dir.is_dir():
        return None
    if template_name.endswith(".gotmpl"):
        candidates = [template_name]
    else:
        candidates = [f"{template_name}.gotmpl", f"{template_name}-chat.gotmpl"]
    for name in candidates:
        path = template_dir / name
        if path.is_file():
            return path.read_text(encoding="utf-8")
    return None


def _get_ollama_chat_template(model: str) -> Optional[str]:
    import ollama

    info = ollama.show(model)
    template = str(info.get("template"))
    return str(template)


def load_ollama_gguf_context(model: str) -> dict:
    manifest_path = _resolve_ollama_manifest(model)
    gguf_path = _resolve_ollama_gguf_blob(manifest_path)
    _assert_gguf_file(gguf_path)

    print("Using Ollama model:", model)
    print("Using manifest:", manifest_path)
    print("Using GGUF:", gguf_path)

    # ---- Embedding-matrix utilities (GGUF) ----
    print("1) Load the GGUF and access the full token embedding matrix (quantized)")
    reader = gguf.GGUFReader(str(gguf_path))

    # token_embd.weight = embedding table (token id -> vector)
    token_tensor = next(t for t in reader.tensors if t.name == "token_embd.weight")
    vocab_size = token_tensor.data.shape[0]

    # Determine embedding dimensionality from a dequantized row (robust across architectures).
    embed_dim = int(
        gguf.dequantize(token_tensor.data[:1], token_tensor.tensor_type).shape[1]
    )

    print(f"Vocab size: {vocab_size}")
    print(f"Embedding dim: {embed_dim}")
    print(f"Embedding quant type: {token_tensor.tensor_type}")

    print("2) Load tokenizer tokens so we can decode token IDs back to text")
    tok_field = reader.fields["tokenizer.ggml.tokens"]
    tokens_raw: Union[List[str], List[bytes]] = tok_field.contents()

    tokens: List[str] = [_tok_to_str(x) for x in tokens_raw]

    chat_template = _get_chat_template_from_gguf(reader)
    if not chat_template:
        print(
            "No chat template found in GGUF; checking for named template to read directly from Ollama source code..."
        )
        template_name = _get_chat_template_name_from_gguf(reader)
        if template_name:
            chat_template = _get_ollama_repo_template(template_name)
    if not chat_template:
        print("No valid chat template found in GGUF; trying Ollama library...")
        chat_template = _get_ollama_chat_template(model)

    return {
        "reader": reader,
        "gguf_path": gguf_path,
        "vocab_size": vocab_size,
        "embed_dim": embed_dim,
        "tokens": tokens,
        "token_tensor": token_tensor,
        "chat_template": chat_template,
    }


# %load_ext line_profiler
# %lprun -f load_ollama_gguf_context context = load_ollama_gguf_context(OLLAMA_MODEL)
context = load_ollama_gguf_context(OLLAMA_MODEL)
print(f"{context['chat_template']=}")


def find_token_id(word: str, tokens: List[str]) -> int:
    """Best-effort lookup for a *single token* ID (not full tokenization)."""
    candidates = [word, "Ġ" + word, "▁" + word, " " + word]
    for c in candidates:
        try:
            return tokens.index(c)
        except ValueError:
            continue
    raise ValueError(
        f"Could not find a single-token match for {word!r}. Try another word."
    )


def dequantize_rows(start: int, end: int, token_tensor) -> np.ndarray:
    """Dequantize embedding rows [start:end) to float32 shape [N, embed_dim]."""
    q = token_tensor.data[start:end]
    return gguf.dequantize(q, token_tensor.tensor_type)


def get_token_vector(token_id: int, token_tensor) -> np.ndarray:
    return dequantize_rows(token_id, token_id + 1, token_tensor)[0]


def get_token_vector_str(token: str) -> np.ndarray:
    tid = find_token_id(token, context["tokens"])
    return get_token_vector(tid, context["token_tensor"])


def normalize(v: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(v)
    return v if n == 0 else (v / n)


def find_closest_tokens(
    vector: np.ndarray,
    vocab_size: int,
    token_tensor,
    tokens: List[str],
    k: int = 5,
    ignore_ids: Optional[Iterable[int]] = None,
    chunk_size: int = 2048,
) -> List[Tuple[str, float, int]]:
    """
    Find top-k cosine-similar tokens across the *entire* vocab by dequantizing in chunks.
    Returns (token_string, similarity, token_id).
    """
    ignore = set(ignore_ids or [])
    target = normalize(vector).astype(np.float32, copy=False)

    heap: List[Tuple[float, int]] = []  # (score, token_id), min-heap

    for start in range(0, vocab_size, chunk_size):
        end = min(start + chunk_size, vocab_size)
        emb = dequantize_rows(start, end, token_tensor)  # float32  [N, embed_dim]
        norms = np.linalg.norm(emb, axis=1, keepdims=True)
        norms[norms == 0] = 1.0
        emb = emb / norms
        scores = emb @ target  # float32 [N]

        for i, score in enumerate(scores):
            tid = start + i
            if tid in ignore:
                continue
            s = float(score)
            if len(heap) < k:
                heapq.heappush(heap, (s, tid))
            elif s > heap[0][0]:
                heapq.heapreplace(heap, (s, tid))

    best = sorted(heap, reverse=True)
    return [(tokens[tid], score, tid) for score, tid in best]


# ---- Chat-like function (local GGUF) ----
# Note: `gguf` provides reading/dequantization utilities but does not execute the model forward pass.
# For inference on a GGUF model we use llama.cpp via `llama-cpp-python`, still pointing at Ollama's local GGUF blob.

SYSTEM_PROMPT = "You are a helpful assistant."

_LLAMA = None


def _extract_system_message(messages: List[dict[str, str]]) -> str:
    for m in messages:
        if m.get("role") == "system":
            return m.get("content", "")
    return ""


def _is_gemma3_template(template: str) -> bool:
    return (
        "<start_of_turn>" in template
        and "$systemPromptAdded" in template
        and ".Messages" in template
    )


def _render_gemma3_template(messages: List[dict[str, str]]) -> str:
    system_text = _extract_system_message(messages)
    usable = [m for m in messages if m.get("role") in ("user", "assistant")]
    parts: List[str] = []
    system_added = False
    for i, m in enumerate(usable):
        last = i == len(usable) - 1
        role = m.get("role")
        content = m.get("content", "")
        if role == "user":
            parts.append("<start_of_turn>user")
            if (not system_added) and system_text:
                system_added = True
                parts.append(system_text)
            parts.append(content)
            parts.append("<end_of_turn>")
            if last:
                parts.append("<start_of_turn>model")
        elif role == "assistant":
            parts.append("<start_of_turn>model")
            parts.append(content)
            if not last:
                parts.append("<end_of_turn>")
    return "\n".join(parts)


def _render_prompt(messages: List[dict[str, str]]) -> str:
    """Render a prompt using GGUF's chat template when available; fallback to a simple ChatML-ish format."""
    chat_template = context["chat_template"]
    if chat_template:
        if _is_gemma3_template(chat_template):
            return _render_gemma3_template(messages)
        try:
            from jinja2 import Environment, StrictUndefined

            def raise_exception(msg: str):
                raise RuntimeError(msg)

            env = Environment(
                undefined=StrictUndefined, trim_blocks=True, lstrip_blocks=True
            )
            j = env.from_string(chat_template)
            return j.render(
                messages=messages,
                add_generation_prompt=True,
                raise_exception=raise_exception,
            )
        except Exception:
            pass

    parts: List[str] = []
    for m in messages:
        parts.append(f"<|im_start|>{m['role']}\n{m['content']}\n<|im_end|>")
    parts.append("<|im_start|>assistant\n")
    return "\n".join(parts)


def _get_llama(gguf_path: Path):
    global _LLAMA
    if _LLAMA is not None:
        return _LLAMA
    try:
        from llama_cpp import Llama
    except Exception as e:
        raise RuntimeError(
            "Missing dependency: llama-cpp-python. Install it in this notebook kernel, e.g.\n"
            "  pip install llama-cpp-python\n\n"
            "This still uses the Ollama-downloaded GGUF file locally and does not use the ollama Python package."
        ) from e

    n_ctx = 4096
    n_threads = max(1, (os.cpu_count() or 4) - 1)
    _LLAMA = Llama(
        model_path=str(gguf_path),
        n_ctx=n_ctx,
        n_threads=n_threads,
        logits_all=False,
        vocab_only=False,
    )
    return _LLAMA


def next_message(user_message: str, messages: List[dict[str, str]]) -> str:
    """
    Chat-like helper similar to `ollama.chat`, but using the local Ollama-downloaded GGUF blob.
    Does not import or call the `ollama` Python package.
    """
    msg = user_message.strip()
    if not msg:
        return ""

    messages.append({"role": "user", "content": msg})
    prompt = _render_prompt(messages)

    llm = _get_llama(context["gguf_path"])
    out = llm.create_completion(
        prompt=prompt,
        max_tokens=256,
        temperature=0.7,
        top_p=0.95,
        stop=["<|im_end|>", "</s>"],
    )
    out = cast(dict, out)
    text = (out.get("choices", [{}])[0].get("text") or "").strip()
    messages.append({"role": "assistant", "content": text})
    return text

Using Ollama model: translategemma:4b
Using manifest: /home/max/.ollama/models/manifests/registry.ollama.ai/library/translategemma/4b
Using GGUF: /home/max/.ollama/models/blobs/sha256-bdbf939b402e2f88fbe3e918beb777813009335756b4c17be7fe008dfe4815d4
1) Load the GGUF and access the full token embedding matrix (quantized)


Exception ignored in: <function LlamaModel.__del__ at 0x7ffa7c9672e0>
Traceback (most recent call last):
  File "/home/max/workspace/github.com/vegerot/localllm/.venv/lib/python3.11/site-packages/llama_cpp/_internals.py", line 86, in __del__
    self.close()
  File "/home/max/workspace/github.com/vegerot/localllm/.venv/lib/python3.11/site-packages/llama_cpp/_internals.py", line 78, in close
    if self.sampler is not None:
       ^^^^^^^^^^^^
AttributeError: 'LlamaModel' object has no attribute 'sampler'


Vocab size: 262208
Embedding dim: 2560
Embedding quant type: 14
2) Load tokenizer tokens so we can decode token IDs back to text
No chat template found in GGUF; checking for named template to read directly from Ollama source code...
No valid chat template found in GGUF; trying Ollama library...
context['chat_template']='{{- range $i, $_ := .Messages }}\n{{- $last := eq (len (slice $.Messages $i)) 1 }}\n{{- if or (eq .Role "user") (eq .Role "system") }}<start_of_turn>user\n{{ .Content }}<end_of_turn>\n{{ if $last }}<start_of_turn>model\n{{ end }}\n{{- else if eq .Role "assistant" }}<start_of_turn>model\n{{ .Content }}{{ if not $last }}<end_of_turn>\n{{ end }}\n{{- end }}\n{{- end }}'


In [9]:
# Example usage of `next_message` (chat-like helper backed by the local GGUF)

messages = [{"role": "system", "content": SYSTEM_PROMPT}]

prompt = "Tell me a joke: "
print("User:", prompt)
print("Assistant:", next_message(prompt, messages))
# print()

# print(
#     "User: Write a Python function that computes cosine similarity between two numpy vectors."
# )
# print(
#     "Assistant:",
#     next_message(
#         "Write a Python function that computes cosine similarity between two numpy vectors.",
#         messages
#     ),
# )
# print()

# print("User: Now explain in 2 bullet points how the prompt is constructed here.")
# print(
#     "Assistant:",
#     next_message(
#         "Now explain in 2 bullet points how the prompt is constructed here.",
#         messages
#     ),
# )

llama_model_loader: loaded meta data with 35 key-value pairs and 883 tensors from /home/max/.ollama/models/blobs/sha256-aeda25e63ebd698fab8638ffb778e68bed908b960d39d0becc650fa981609d25 (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                gemma3.attention.head_count u32              = 8
llama_model_loader: - kv   1:             gemma3.attention.head_count_kv u32              = 4
llama_model_loader: - kv   2:                gemma3.attention.key_length u32              = 256
llama_model_loader: - kv   3:            gemma3.attention.sliding_window u32              = 1024
llama_model_loader: - kv   4:              gemma3.attention.value_length u32              = 256
llama_model_loader: - kv   5:                         gemma3.block_count u32              = 34
llama_model_loader: - kv   6:                      gemma3.context_length u32              = 131072
llama_model_loader:

User: Tell me a joke: 


llama_model_load: error loading model: error loading model hyperparameters: key not found in model: gemma3.attention.layer_norm_rms_epsilon
llama_model_load_from_file_impl: failed to load model


ValueError: Failed to load model from file: /home/max/.ollama/models/blobs/sha256-aeda25e63ebd698fab8638ffb778e68bed908b960d39d0becc650fa981609d25

In [ ]:
# 3) Demo: king - man + woman (single-token lookup)
print("3) Demo")
print("3a) find token IDs for inputs")
king_id = find_token_id("king", context["tokens"])
man_id = find_token_id("man", context["tokens"])
woman_id = find_token_id("woman", context["tokens"])

king_vec = get_token_vector(king_id, context["token_tensor"])
queen_vec = get_token_vector(
    find_token_id("queen", context["tokens"]), context["token_tensor"]
)
boy_vec = get_token_vector(
    find_token_id("boy", context["tokens"]), context["token_tensor"]
)

man_vec = get_token_vector(man_id, context["token_tensor"])
woman_vec = get_token_vector(woman_id, context["token_tensor"])
masculinity_vector = man_vec - woman_vec

hebrew_vector = get_token_vector_str("近平") - get_token_vector_str(
    "Jinping"
)  # + get_token_vector_str("apple", context['tokens'], context['token_tensor']) #+ get_token_vector_str("Playstation", context['tokens'], context['token_tensor'])

target_vector = get_token_vector_str("president") + hebrew_vector

ignore = []
print("3b) find closest tokens")
matches = find_closest_tokens(
    target_vector,
    k=25,
    ignore_ids=ignore,
    vocab_size=context["vocab_size"],
    token_tensor=context["token_tensor"],
    tokens=context["tokens"],
)

print("\nClosest matches in the entire vocabulary:")
for i, (tok, score, tid) in enumerate(matches, start=1):
    print(f"{i}. {tok!r} (id={tid}, similarity={score:.4f})")

longest_token = max(context["tokens"], key=lambda t: len(t))
print(
    f"\nLongest token in vocabulary is {len(longest_token)} characters: {longest_token!r}"
)

ten_longest_tokens = heapq.nlargest(10, context["tokens"], key=lambda t: len(t))
print("\nTop 10 longest tokens in vocabulary:")
for i, tok in enumerate(ten_longest_tokens, start=1):
    print(f"{i}. {len(tok)} chars: {tok!r}")

3) Demo
3a) find token IDs for inputs
3b) find closest tokens

Closest matches in the entire vocabulary:
1. 'president' (id=59692, similarity=0.6476)
2. '▁president' (id=6207, similarity=0.5356)
3. '近平' (id=145720, similarity=0.5196)
4. 'President' (id=27665, similarity=0.4860)
5. '▁President' (id=5376, similarity=0.4747)
6. '▁presidente' (id=28769, similarity=0.4370)
7. '▁PRESIDENT' (id=124009, similarity=0.4338)
8. '▁presidents' (id=81965, similarity=0.3999)
9. '▁président' (id=59687, similarity=0.3914)
10. '▁президента' (id=163921, similarity=0.3877)
11. '▁prezident' (id=180283, similarity=0.3824)
12. '▁президен' (id=210929, similarity=0.3786)
13. '▁президент' (id=115284, similarity=0.3729)
14. 'presidente' (id=120343, similarity=0.3694)
15. '▁Präsident' (id=126714, similarity=0.3691)
16. '总统' (id=115884, similarity=0.3649)
17. '▁presiden' (id=205109, similarity=0.3618)
18. '▁প্রেসিডেন্ট' (id=39010, similarity=0.3544)
19. '▁Presiden' (id=108539, similarity=0.3470)
20. '▁presidenta' 